In [4]:
import sys
!{sys.executable} -m pip install torch==1.5.1+cpu torchvision==0.6.1+cpu -f https://download.pytorch.org/whl/torch_stable.html
!{sys.executable} -m pip install --user albumentations

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cpu/torch-1.5.1%2Bcpu-cp37-cp37m-win_amd64.whl (109.2 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.6.1%2Bcpu-cp37-cp37m-win_amd64.whl (456 kB)
Processing c:\users\94201\appdata\local\pip\cache\wheels\97\ec\48\0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0\imgaug-0.2.6-cp37-none-any.whl


In [5]:
import cv2
import os.path
import json
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image, ImageDraw
%matplotlib inline

In [6]:
class InteriorLandmarksDataset(Dataset):

    def __init__(self, json_file, img_dir, segm_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        with open(json_file) as f:
            self.landmarks = json.load(f)
        self.img_dir = img_dir
        self.segm_dir = segm_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks)
    
    def displayOnCV2(self, image):
        image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        cv2.imshow('img',image)

    def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()

        img_name = os.path.join(self.img_dir,
                                self.landmarks[idx]["file"])
        segm_name = os.path.join(self.segm_dir,
                                self.landmarks[idx]["file"])

        image = Image.open(img_name)
        
        try:
            segm = Image.open(segm_name)
            segm = Image.fromarray(np.asarray(segm)[:, :, 0])
            assert segm is None
        except:
            landmarks = self.landmarks[idx]["points"]
            
            landmarks = [tuple(p) for p in landmarks]
            segm = Image.fromarray(np.zeros_like(np.asarray(image))[:, :, 0])
            imgDraw = ImageDraw.Draw(segm)
            if landmarks:
                imgDraw.polygon(landmarks, fill=255, outline = None)
            
            # TODO: move saving to ImageService
            segm.save(segm_name)
        
        sample = {'image': np.asarray(image), 'mask': np.asarray(segm)}
        if self.transform:
            sample = self.transform(**sample)
            
        return sample

In [7]:
class RandomResizedCrop:
    def __init__(self, height, width, part=0.9):
        pass
    
    def __call__(self, image, mask):
        xmin, xmax, ymin, ymax = 0,0,0,0 # TODO: randomize
        # check if squuare_crop / square > self.part is enough
        
        image = image[ymin:ymax, xmin:xmax, :]
        mask = mask[ymin:ymax, xmin:xmax, :]
        return image, mask
    
class RandomFlip:
    def __init__(self, p):
        self.p = p
    
    def __call__(self, image, mask):
        pass

In [8]:
import albumentations
from albumentations.pytorch.transforms import ToTensor
T = albumentations.Compose([
    albumentations.RandomResizedCrop(256, 256, 
                                    scale=(0.8, 1.0), ratio=(0.75, 1.33), 
                                                              interpolation=1, 
                                                              always_apply=True, 
                                                              p=1.0),
    albumentations.HorizontalFlip(),
    ToTensor(num_classes=1, sigmoid=False, normalize={'mean': [0,0,0], 'std': [1,1,1]}) # TODO: get normalize ImageService 
])

T_test = albumentations.Compose([
    albumentations.Resize(256, 256),
    ToTensor(num_classes=1, sigmoid=False, normalize={'mean': [0,0,0], 'std': [1,1,1]})
])

In [9]:
datasetLearn = InteriorLandmarksDataset('data/answersLearn.json', 'data/imgLearn', 'data/imgSegmLearn', transform=T)
datasetTest = InteriorLandmarksDataset('data/answersTest.json', 'data/imgTest', 'data/imgSegmTest', transform=T_test)

In [10]:
# T = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop(256),
#     transforms.ToTensor(),
# ])

In [11]:
# datasetLearn = InteriorLandmarksDataset('data/answersLearn.json', 'data/imgLearn', T)
# datasetTest = InteriorLandmarksDataset('data/answersTest.json', 'data/imgTest', T)

In [12]:
# sample = next(iter(datasetLearn))

In [13]:
# sample['image'].shape

In [14]:
def vis_torch(sample):
    I = sample['image'].numpy().copy()
    
    gt = sample['segm'].numpy().copy()
    I[gt > 0] = 255
    plt.imshow(gt[0])


def vis(sample):
    I = np.asarray(sample['image']).copy()
    
    gt = np.asarray(sample['segm']).copy()
    I[gt > 0] = 255
    
#     for x, y in sample['landmarks']:
#         x, y = int(x), int(y)
#         I[y:y+10, x:x+10] = 0
    plt.imshow(gt)
#vis_torch(sample)

In [15]:
trainloader = torch.utils.data.DataLoader(datasetLearn, batch_size=4, num_workers=0,
                                          shuffle=True)


Let's play with dataloader and model

In [16]:
minibatch = next(iter(trainloader))

FileNotFoundError: [Errno 2] No such file or directory: 'data/imgLearn\\4c64410fd14f4d5e203245da252712c7.jpg'

In [17]:
from model import UNet
model = UNet(1)

In [18]:
# %%timeit
pred = model(minibatch['image'])

NameError: name 'minibatch' is not defined

In [19]:
plt.imshow(pred[0, 0].detach().numpy())

NameError: name 'pred' is not defined

In [20]:
from model import UNet
import torch.nn as nn
import torch.optim as optim
model = UNet(1)
loss = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005)

num_epochs = 50
for epoch in range(num_epochs):
    trainloader = torch.utils.data.DataLoader(datasetLearn, batch_size=4, num_workers=0,
                                          shuffle=True)
    testloader = torch.utils.data.DataLoader(datasetTest, batch_size=4, num_workers=0,
                                          shuffle=True)
    
    i = 0
    for minibatch in trainloader:
        inp = minibatch['image']
        gt = minibatch['mask']
        #ValueError: Target size (torch.Size([4, 3, 256, 256])) must be the same as input size (torch.Size([4, 1, 256, 256]))
        pred = model(inp)
        loss_ = loss(pred, gt)
        print(loss_.item())
        loss_.backward()
        optimizer.step()
        i += 1
        if i > 100000:
            break
            
    break

0.7060145139694214


FileNotFoundError: [Errno 2] No such file or directory: 'data/imgLearn\\8.jpg'

In [ ]:
class Deploy:
    def __init__(self, model, T):
        self.model = model
        self.T = T
        
    def __call__(self, image):
        '''
        RGB numpy uint8 array
        '''
        preprocessed = self.T(image=image)
        pred = model(preprocessed['image'].unsqueeze(0))[0, 0]
        pred = pred.detach().cpu().numpy()
        print(np.unique(pred))
        print(pred.shape)
        pred_resize = cv2.resize(pred, (image.shape[1], image.shape[0]))
        I = image.copy()
        I[pred_resize > 0.5, 0] = 255
        return I
    
deployment = Deploy(model, T_test)

image = np.asarray(Image.open('data/imgTestSrc/IMG_1102 cor.jpg'))
print(image.shape)
plt.imshow(image)
pred = deployment(image)
plt.imshow(pred)

In [ ]:
minibatch['idx']

In [ ]:
torch.save(model, 'model.pth')

In [ ]:
model = torch.load('model.pth')